# Vehicle detection

In [1]:
from sklearn import svm, grid_search
import time
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import glob
import cv2
from skimage.feature import hog
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import random
import pickle
from sklearn.externals import joblib 
from scipy.ndimage.measurements import label
from moviepy.editor import VideoFileClip
from IPython.display import HTML

C:\Users\Wyatt\Miniconda3\envs\carnd-term1\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\Wyatt\Miniconda3\envs\carnd-term1\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
def cal_undistort(image, img_points, obj_points):
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(obj_points, img_points, gray.shape[::-1], None, None)
    
    return cv2.undistort(image, mtx, dist, None, mtx)

## transform the pictures and save it

def wrap(img):
    src = np.float32([[190,720], [586,455], [696,455], [1130,720]])
    dst = np.float32([[240,720], [240,0], [1040,0], [1040, 720]])
    M = cv2.getPerspectiveTransform(src, dst)
    img_size = (img.shape[1], img.shape[0])
    wraped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
    return wraped


def hls_pip(img, thres_s=(0,255), thres_h = (0,255), thres_l=(0,255)):
    image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    H = image[:,:,0]
    S = image[:,:,2]
    L = image[:,:,1]
    
    binary_output = np.zeros_like(S)
    binary_output[((S<thres_s[1]) & (S>thres_s[0]))|((H<thres_h[1]) & (H>thres_h[0]))|((L<thres_l[1]) & (L>thres_l[0]))] = 1
    
    return binary_output

In [3]:
def histogram_find(nwindows=9, margin=100, minpix=50, plot=0, save=0, image=None):
    
    binary_wraped = image
    
    # take a histogram of bottom half of the image
    histogram = np.sum(binary_wraped[binary_wraped.shape[0]//2:,:], axis=0)
    
    # generate a picture to visualize the result
    out_image = np.dstack((binary_wraped,binary_wraped,binary_wraped))
    
    # find the peak of the histogram as the starting point
    mid_point = (500,780)
    leftx_start = np.argmax(histogram[:mid_point[0]])
    rightx_start = np.argmax(histogram[mid_point[1]:]) + mid_point[1]
    
    #set the height of the sliding window
    window_height = np.int(histogram.shape[0]//nwindows)
    
    # find all the nonzero pixels
    nonzero = binary_wraped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # define the current position
    leftx_current = leftx_start
    rightx_current = rightx_start
    
    # list to save the left and right positions of pixels in the form of index
    left_lane_inds = []
    right_lane_inds = []
    
    # slide the window
    for window in range(nwindows):
        # define the window boundaries
        win_y_low = binary_wraped.shape[0] - (1 + window) * window_height
        win_y_high = binary_wraped.shape[0] - window * window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # visualize the window
        if plot:
            cv2.rectangle(out_image,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
            cv2.rectangle(out_image,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2)
        
        # find the nonzero pixel within the window in x and y
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        
        # add the index into the list
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # relocate the center of the window if find > minpix pixels
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
            
    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # Visualization
    if plot:
        ploty = np.linspace(0, binary_wraped.shape[0]-1, binary_wraped.shape[0] )
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

        out_image[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
        out_image[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [255, 0, 0]
        plt.figure()
        plt.imshow(out_image)
        plt.plot(left_fitx, ploty, color='yellow')
        plt.plot(right_fitx, ploty, color='yellow')
        plt.xlim(0, 1280)
        plt.ylim(720, 0)
        plt.title(figurename, fontsize=50)
    
    # save the picture
    if save:
        plt.savefig('./output_images/histogram/'+figurename)
    return left_fit, right_fit

In [4]:
def precious_search(left_fit, right_fit,margin=100, plot=0, save=0, image=None):
    
    binary_wraped = image
    nonzero = binary_wraped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    left_lane_inds = ((nonzerox > (left_fit[0] * nonzeroy ** 2 + left_fit[1]*nonzeroy + left_fit[2] - margin)) & 
                     (nonzerox < (left_fit[0] * nonzeroy ** 2 + left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0] * nonzeroy ** 2 + right_fit[1]*nonzeroy + right_fit[2] - margin)) & 
                     (nonzerox < (right_fit[0] * nonzeroy ** 2 + right_fit[1]*nonzeroy + right_fit[2] + margin)))
    
    leftx = nonzerox[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    ploty = np.linspace(0, binary_wraped.shape[0]-1, binary_wraped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    ## visualization
    out_img = np.dstack((binary_wraped, binary_wraped, binary_wraped))
    window_img = np.zeros_like(out_img)
    
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [255, 0, 0]

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                                  ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                                  ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Visualization
    if plot:
        plt.figure()
        cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
        cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
        result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
        plt.imshow(result)
        plt.plot(left_fitx, ploty, color='yellow')
        plt.plot(right_fitx, ploty, color='yellow')
        plt.xlim(0, 1280)
        plt.ylim(720, 0)
        plt.title(figurename, fontsize=50)
        
    if save:
        plt.savefig('./output_images/histogram/precious_'+figurename)
    
    return left_fit, right_fit

In [5]:
def cal_curv(left_fit, right_fit):
    
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
    # calculate the curvature around the car position(720 pixel)
    left_curv = ((1 + (2*left_fit[0]*720*ym_per_pix + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
    right_curv = ((1 + (2*right_fit[0]*720*ym_per_pix + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])
    
    return left_curv, right_curv

In [6]:
class Line():
    def __init__(self, n):
        # was the line detected in the last iteration?
        self.detected = False
        # set of the x values of fitted line over the last n iterations
        self.setx = []
        # average x values of the fitted line over the last n iterations
        self.iter_num = n
        self.bestx = None  
        # set of polynomial coefficients over the last n iterations
        self.set_poly = []
        # polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        # radius of curvature of the line in some units
        self.radius_of_curvature = None 
        # distance in meters of vehicle center from the line
        self.line_base_pos = None 
        # count the number of frame
        self.count = 0
        # strings for printing
        self.str1 = None
        self.str2 = None
        
    def update(self, fitline, curvature):
        self.count += 1
        ploty =  np.linspace(720,0, 720)
        self.radius_of_curvature = curvature
        
        if self.detected:
            
            if self.count <= self.iter_num:
                self.setx.append(self.best_fit[0] * ploty ** 2 + self.best_fit[1]*ploty + self.best_fit[2])
                self.set_poly.append(fitline)
            else:
                del self.setx[0]
                del self.set_poly[0]
                self.setx.append(self.best_fit[0] * ploty ** 2 + self.best_fit[1]*ploty + self.best_fit[2])
                self.set_poly.append(fitline)
            sumx = 0
            for i in range(len(self.setx)):
                sumx += self.setx[i]
            self.bestx = sumx/len(self.setx)
            sumpoly = 0
            for i in range(len(self.set_poly)):
                sumpoly += self.set_poly[i]
            self.best_fit = sumpoly/len(self.set_poly)
            
            self.line_base_pos = self.bestx[0] - 640
            
        else:
            self.detected = True
            self.set_poly.append(fitline)
            self.best_fit = fitline
            self.bestx = self.best_fit[0] * ploty ** 2 + self.best_fit[1]*ploty + self.best_fit[2]
            self.setx.append(self.best_fit[0] * ploty ** 2 + self.best_fit[1]*ploty + self.best_fit[2])
            self.line_base_pos = self.bestx[0] - 640

In [7]:
# define a function to check out wether the new result good or not
def check_output(left_fit, right_fit, oldleft = None, oldright = None, first = False):
    result = True
    ploty =  np.linspace(720,0,720)
    left_x = left_fit[0] * ploty ** 2 + left_fit[1]*ploty + left_fit[2]
    right_x = right_fit[0] * ploty ** 2 + right_fit[1]*ploty + right_fit[2]
    
    # checking that the left and right lane line are roughly parallel
    fit_diff = [np.abs(left_fit[0] - right_fit[0])/left_fit[0], np.abs(left_fit[1] - right_fit[1])/left_fit[1]]
    
    if fit_diff[0] > 0.5 or fit_diff[1] > 0.5:
        result = False
    if not first:
        # calculate the difference of polyfit parameters
        poly_diff_left = (oldleft.best_fit - left_fit) / oldleft.best_fit
        poly_diff_right = (oldright.best_fit - right_fit) / oldright.best_fit
        
        if not (abs(poly_diff_left)<np.array([0.7,0.5,0.2])).all() or not (abs(poly_diff_right)<np.array([0.7,0.5,0.2])).all():
                result = False
                
    return result

In [8]:
def pro_video(image, Left, Right, img_points, obj_points):
    
    S_thres = (200,255)
    H_thres = (22,24)
    L_thres = (170,255)

    good = True
    src = np.float32([[190,720], [586,455], [696,455], [1130,720]])
    dst = np.float32([[240,720], [240,0], [1040,0], [1040, 720]])
    Minv = cv2.getPerspectiveTransform(dst, src)
    
    # undistort the image
    undist = cal_undistort(image, img_points, obj_points)
    
    # pre-process the image and tranform into bird-eye view
    binary_output = hls_pip(undist, thres_s=S_thres, thres_h=H_thres,thres_l=L_thres)
    image = wrap(binary_output)*255
    
    # find the lane line
    if Left.detected:
        left_fit, right_fit = precious_search(Left.best_fit, Right.best_fit, image=image)
        if not check_output(left_fit, right_fit, Left, Right):
            left_fit, right_fit = histogram_find(image=image)
            if not check_output(left_fit, right_fit, Left, Right, first = True):
                good = False
    else:
        left_fit, right_fit = histogram_find(image=image)
    
    if good:
        # calculate the curvate 
        left_cur, right_cur = cal_curv(left_fit, right_fit)

        # update the information in the class
        Left.update(left_fit, left_cur)
        Right.update(right_fit, right_cur)
    
    # take the average information in the class
    left_fit = Left.best_fit
    right_fit = Right.best_fit
    
    y_array = np.linspace(720,0, 720)
    leftx = left_fit[0] * y_array ** 2 + left_fit[1]*y_array + left_fit[2]
    rightx = right_fit[0] * y_array ** 2 + right_fit[1]*y_array + right_fit[2]
    
    # Create an image to draw the lines on
    warp_color = np.zeros_like(undist).astype(np.uint8)

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([leftx, y_array]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([rightx, y_array])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(warp_color, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(warp_color, Minv, (image.shape[1], image.shape[0])) 
    
    # Combine the result with the original image 
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    
    # print the curvatue on the image every 5 frame
    font = cv2.FONT_HERSHEY_SIMPLEX
    if (Left.count-1) % 5 == 0:
        curc = round(0.5*(Left.radius_of_curvature + Right.radius_of_curvature)/1000, 2)
        Left.str1 = 'The curvate of the lane line: ' + str(curc) + ' km'

        # find the position of the car
        right_line_base = rightx[0]
        left_line_base = leftx[0]
        middle = 640
        distance = 0.5 * (Left.line_base_pos + Right.line_base_pos)
        distance = round(distance * 3.7/700 * 100, 2)
        if distance < 0:
            Left.str2 = 'right of the center: ' + str(abs(distance)) + ' cm'
        else:
            Left.str2 = 'left of the center: ' + str(abs(distance)) + ' cm'
            
    cv2.putText(result,Left.str1,(400,650), font, 1,(0,0,255),2,cv2.LINE_AA)
    cv2.putText(result,Left.str2,(400,700), font, 1,(0,0,255),2,cv2.LINE_AA)
    
    return result

In [9]:
## define a function to find hog features of the images
## set vis as true to show the result.

def get_hog_features(img, orient, pix_per_cell, cell_per_block, vis=False,
                     feature_vec=True):
    return_list = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block),
                                  block_norm= 'L2-Hys', transform_sqrt=False, 
                                  visualise= vis, feature_vector= feature_vec)
    
    # name returns explicitly
    if vis:
        hog_features = return_list[0]
        hog_image = return_list[1]
        return hog_features, hog_image
    else:
        return return_list

In [10]:
## combine the features in color space

# Define a function to compute binned color features  
def bin_spatial(img, size=(32, 32)):
    # Use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(img, size).ravel() 
    # Return the feature vector
    return features

# Define a function to compute color histogram features  
def color_hist(img, nbins=32, bins_range=(0, 256)):
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

In [11]:
## combine the features in hog, spatial and in color histogramm

def extract_features(imgs, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for image in imgs:
        file_features = []
        # apply color conversion if other than 'RGB'
        if color_space != 'RGB':
            string = 'cv2.COLOR_RGB2' + color_space
            feature_image = cv2.cvtColor(image, eval(string))
        else: feature_image = np.copy(image)      

        if spatial_feat == True:
            spatial_features = bin_spatial(feature_image, size=spatial_size)
            file_features.append(spatial_features)
        if hist_feat == True:
            # Apply color_hist()
            hist_features = color_hist(feature_image, nbins=hist_bins)
            file_features.append(hist_features)
        if hog_feat == True:
        # Call get_hog_features() with vis=False, feature_vec=True
            if hog_channel == 'ALL':
                hog_features = []
                for channel in range(feature_image.shape[2]):
                    hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                        orient, pix_per_cell, cell_per_block, 
                                        vis=False, feature_vec=True))
                hog_features = np.ravel(hog_features)        
            else:
                hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                            pix_per_cell, cell_per_block, vis=False, feature_vec=True)
            # Append the new feature vector to the features list
            file_features.append(hog_features)
        features.append(np.concatenate(file_features))
    # Return list of feature vectors
    return features

In [12]:
## define a function to draw the boxes on picture for visualizing the sliding windows

def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    # Make a copy of the image
    imcopy = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
    # Return the image copy with boxes drawn
    return imcopy

In [18]:
## function to produce heatmap
def add_heat(heatmap, bbox_list):
    # Iterate through list of bboxes
    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1

    # Return updated heatmap
    return heatmap# Iterate through list of bboxes
    
def apply_threshold(heatmap, threshold):
    # Zero out pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    # Return thresholded map
    return heatmap

def draw_labeled_bboxes(img, labels):
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        # filt out the wired window
        long = np.max(nonzerox) - np.min(nonzerox)
        width = np.max(nonzeroy) - np.min(nonzeroy)
        if long*1.3 <= width:
            continue
        # Draw the box on the image
        else: cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 6)
    # Return the image
    return img

In [24]:
# Define a function that can extract features using sub-sampling and make predictions
def find_windows(img, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins):
    
    windows = []
    
    draw_img = np.copy(img)
    img = img.astype(np.float32)/255
    
    img_tosearch = img[ystart:ystop,:,:]
    if scale != 1:
        imshape = img_tosearch.shape
        img_tosearch = cv2.resize(img_tosearch, (np.int(imshape[1]/scale), np.int(imshape[0]/scale)))

    # Define blocks and steps as above
    nxblocks = (img_tosearch[:,:,0].shape[1] // pix_per_cell) - cell_per_block + 1
    nyblocks = (img_tosearch[:,:,0].shape[0] // pix_per_cell) - cell_per_block + 1 
    nfeat_per_block = orient*cell_per_block**2
    
    # 64 was the orginal sampling rate, with 8 cells and 8 pix per cell
    window = 64
    nblocks_per_window = (window // pix_per_cell) - cell_per_block + 1
    cells_per_step = 2  # Instead of overlap, define how many cells to step
    nxsteps = (nxblocks - nblocks_per_window) // cells_per_step + 1
    nysteps = (nyblocks - nblocks_per_window) // cells_per_step + 1
    
    for xb in range(nxsteps):
        for yb in range(nysteps):
            ypos = yb*cells_per_step
            xpos = xb*cells_per_step
            xleft = xpos*pix_per_cell
            ytop = ypos*pix_per_cell

            # Extract the image patch
            subimg = cv2.resize(img_tosearch[ytop:ytop+window, xleft:xleft+window], (64,64))
          
            # Get features
            features = extract_features([subimg], color_space='YUV', 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel='ALL', spatial_feat=True, 
                        hist_feat=True, hog_feat=True)

            # Scale features and make a prediction
            test_features = X_scaler.transform(features)    
            #test_features = X_scaler.transform(np.hstack((shape_feat, hist_feat)).reshape(1, -1))    
            test_prediction = svc.predict(test_features)
            
            if test_prediction == 1 and svc.decision_function(test_features) >= 0.5:
                xbox_left = np.int(xleft*scale)
                ytop_draw = np.int(ytop*scale)
                win_draw = np.int(window*scale)
                windows.append([(xbox_left, ytop_draw+ystart), (xbox_left+win_draw, ytop_draw+win_draw+ystart)]) 
                
    return windows

In [25]:
## define a class to save the last n frames
class save_frame():
    def __init__(self, n):
        self.old_windows = [[] for i in range(n//2)] 
        self.max_frames = n
        
    def update_window(self, windows): # update the previous windows and throw some old windows
        self.old_windows.append(windows)
        if len(self.old_windows) > self.max_frames:
            self.old_windows = self.old_windows[len(self.old_windows)-self.max_frames:] 

In [26]:
def find_video_withlines(image):
    
    global old_frames, Left, Right, clf, X_scaler 

    ## set parameters
    color_space = 'YUV' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
    orient = 11  # HOG orientations
    pix_per_cell = 16 # HOG pixels per cell
    cell_per_block = 2 # HOG cells per block
    hog_channel = 'ALL' # Can be 0, 1, 2, or "ALL"
    spatial_size = (16, 16) # Spatial binning dimensions
    hist_bins = 16    # Number of histogram bins
    
    windows = []
    for scale, ystart, ystop in [(1, 400, 464), (1, 416, 480), (1, 380, 480), (1.2, 375, 495),
                                 (1.3, 390, 480), (1.4, 390, 480), (1.5, 400, 496), 
                                 (1.5, 416, 540), (1.7, 400, 510), (2, 400, 560)]:
        out_img = find_windows(image, ystart, ystop, scale, clf, X_scaler, orient, 
                               pix_per_cell, cell_per_block, spatial_size, hist_bins)
        windows.extend(out_img)
    
    old_frames.update_window(windows)
    heat = np.zeros_like(image[:,:,0]).astype(np.float)
    heat = add_heat(heat,windows)
    for old_frame_windows in old_frames.old_windows:
        heat = add_heat(heat,old_frame_windows)
    heat = apply_threshold(heat,2*(2+len(old_frames.old_windows)))
    heatmap = np.clip(heat, 0, 255)
    
    labels = label(heatmap)
    
    ## draw the lines at relevant information on the video
    image_to_draw = pro_video(image, Left, Right, img_points, obj_points)
    
    draw_img = draw_labeled_bboxes(image_to_draw, labels)
    
    return draw_img

In [27]:
## define the function tp operate on videos

def find_video(image):
    
    global old_frames, clf, X_scaler

    ## set parameters
    color_space = 'YUV' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
    orient = 11  # HOG orientations
    pix_per_cell = 16 # HOG pixels per cell
    cell_per_block = 2 # HOG cells per block
    hog_channel = 'ALL' # Can be 0, 1, 2, or "ALL"
    spatial_size = (16, 16) # Spatial binning dimensions
    hist_bins = 16    # Number of histogram bins
    
    windows = []
    for scale, ystart, ystop in [(1, 400, 464), (1, 416, 480), (1, 380, 480), (1.2, 375, 495),
                                 (1.3, 390, 480), (1.4, 390, 480), (1.5, 400, 496), 
                                 (1.5, 416, 540), (1.7, 400, 510), (2, 400, 560)]:
        out_img = find_windows(image, ystart, ystop, scale, clf, X_scaler, orient,
                               pix_per_cell, cell_per_block, spatial_size, hist_bins)
        windows.extend(out_img)
    
    old_frames.update_window(windows)
    heat = np.zeros_like(image[:,:,0]).astype(np.float)
    for old_frame_windows in old_frames.old_windows:
        heat = add_heat(heat,old_frame_windows)
    heat = apply_threshold(heat,2*(2+len(old_frames.old_windows)))
    heatmap = np.clip(heat, 0, 255)
    
    labels = label(heatmap)
    draw_img = draw_labeled_bboxes(np.copy(image), labels)
    
    return draw_img

In [28]:
Left = Line(5)
Right = Line(5)

## load the data
clf = joblib.load("./models/model_YUV.m")
with open('./models/data_YUV.pickle', 'rb') as file:
    data_dict = pickle.load(file)
    X_scaler = data_dict['X_scaler']

with open('calibration.pickle', 'rb') as file:
    data_dict = pickle.load(file)
img_points = data_dict['img']
obj_points = data_dict['obj']

In [22]:
old_frames = save_frame(10)
myclip = VideoFileClip('test_video.mp4')
clip = myclip.fl_image(find_video)
%time clip.write_videofile('./output_videos/test_video.mp4', audio=False)

[MoviePy] >>>> Building video ./output_videos/test_video.mp4
[MoviePy] Writing video ./output_videos/test_video.mp4


 97%|███████████████████████████████████████████████████████████████████████████████▉  | 38/39 [00:25<00:00,  1.48it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./output_videos/test_video.mp4 

Wall time: 27.3 s


In [23]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format('output_videos/test_video.mp4'))

In [30]:
old_frames = save_frame(10)
myclip = VideoFileClip('project_video.mp4')
clip = myclip.fl_image(find_video)
%time clip.write_videofile('./output_videos/project_video.mp4', audio=False)

[MoviePy] >>>> Building video ./output_videos/project_video.mp4
[MoviePy] Writing video ./output_videos/project_video.mp4


100%|█████████████████████████████████████████████████████████████████████████████▉| 1260/1261 [15:26<00:00,  1.36it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./output_videos/project_video.mp4 

Wall time: 15min 27s


In [31]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format('output_videos/project_video.mp4'))

In [17]:
old_frames = save_frame(10)
myclip = VideoFileClip('test_video.mp4')
clip = myclip.fl_image(find_video_withlines)
%time clip.write_videofile('./output_videos/test_video_withline.mp4', audio=False)

[MoviePy] >>>> Building video ./output_videos/test_video_withline.mp4
[MoviePy] Writing video ./output_videos/test_video_withline.mp4


 97%|███████████████████████████████████████████████████████████████████████████████▉  | 38/39 [14:12<00:22, 22.44s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./output_videos/test_video_withline.mp4 

Wall time: 14min 14s


In [18]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format('output_videos/test_video_withline.mp4'))

In [23]:
old_frames = save_frame(10)
myclip = VideoFileClip('project_video.mp4')
clip = myclip.fl_image(find_video_withlines)
%time clip.write_videofile('./output_videos/project_video_withline.mp4', audio=False)

[MoviePy] >>>> Building video ./output_videos/project_video_withline.mp4
[MoviePy] Writing video ./output_videos/project_video_withline.mp4


  0%|                                                                               | 1/1261 [00:21<7:22:00, 21.05s/it]

KeyboardInterrupt: 

In [24]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format('output_videos/project_video_withline.mp4'))

In [29]:
def output_video_withlines(image):
    
    global old_frames, clf, X_scaler, windows_for_draw

    ## set parameters
    color_space = 'YUV' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
    orient = 11  # HOG orientations
    pix_per_cell = 16 # HOG pixels per cell
    cell_per_block = 2 # HOG cells per block
    hog_channel = 'ALL' # Can be 0, 1, 2, or "ALL"
    spatial_size = (16, 16) # Spatial binning dimensions
    hist_bins = 16    # Number of histogram bins
    
    windows = []
    for scale, ystart, ystop in [(1, 400, 464), (1, 416, 480), (1, 380, 480), (1.2, 375, 495),
                                 (1.3, 390, 480), (1.4, 390, 480), (1.5, 400, 496), 
                                 (1.5, 416, 540), (1.7, 400, 510), (2, 400, 560)]:
        out_img = find_windows(image, ystart, ystop, scale, clf, X_scaler, orient,
                               pix_per_cell, cell_per_block, spatial_size, hist_bins)
        windows.extend(out_img)
    
    old_frames.update_window(windows)
    heat = np.zeros_like(image[:,:,0]).astype(np.float)
    for old_frame_windows in old_frames.old_windows:
        heat = add_heat(heat,old_frame_windows)
    heat = apply_threshold(heat,2*(2+len(old_frames.old_windows)))
    heatmap = np.clip(heat, 0, 255)
    
    labels = label(heatmap)
    
    temp_windows = []
    
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        # filt out the wired window
        long = np.max(nonzerox) - np.min(nonzerox)
        width = np.max(nonzeroy) - np.min(nonzeroy)
        if long*1.3 <= width:
            continue
        # Draw the box on the image
        else: temp_windows.append((bbox[0], bbox[1]))
    
    windows_for_draw.append(temp_windows)
    
    return image

In [30]:
windows_for_draw = []
old_frames = save_frame(10)
myclip = VideoFileClip('project_video.mp4')
clip = myclip.fl_image(output_video_withlines)
%time clip.write_videofile('./output_videos/template.mp4', audio=False)

[MoviePy] >>>> Building video ./output_videos/template.mp4
[MoviePy] Writing video ./output_videos/template.mp4


100%|█████████████████████████████████████████████████████████████████████████████▉| 1260/1261 [41:24<00:01,  1.97s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./output_videos/template.mp4 

Wall time: 41min 26s


In [31]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format('output_videos/template.mp4'))

In [32]:
def draw_all_together(image):
    global index
    
    for bboxs in windows_for_draw[index]:
        cv2.rectangle(image, bboxs[0], bboxs[1], (0,0,255), 6)
    index += 1
    
    return image

In [33]:
data = {'windows_for_draw': windows_for_draw}
with open('boxs_data.pickle', 'wb') as file:
    pickle.dump(data,file)

In [34]:
index = 0
with open('boxs_data.pickle', 'rb') as file:
    data = pickle.load(file)
    windows_for_draw = data['windows_for_draw']
myclip = VideoFileClip('video_from_advanced.mp4')
clip = myclip.fl_image(draw_all_together)
%time clip.write_videofile('./output_videos/project_video_withline.mp4', audio=False)

[MoviePy] >>>> Building video ./output_videos/project_video_withline.mp4
[MoviePy] Writing video ./output_videos/project_video_withline.mp4


100%|█████████████████████████████████████████████████████████████████████████████▉| 1260/1261 [01:21<00:00, 15.55it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./output_videos/project_video_withline.mp4 

Wall time: 1min 23s


In [35]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format('output_videos/project_video_withline.mp4'))